In [10]:
%matplotlib inline
%matplotlib nbagg
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import prettytensor as pt

In [11]:
rows, cols = 70, 100
img_shape = (rows, cols)
img_flat_size = rows * cols
img_classes = 3
img_channels = 3

In [12]:
resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"

In [13]:
isic_pickle_file = resized_folder_path + "ISIC.pickle"
with open(isic_pickle_file, "rb") as f:
    save = pickle.load(f)
    train_dataset =  save["train_dataset"]
    train_labels =  save["train_labels"]
    test_dataset =  save["test_dataset"]
    test_labels =  save["test_labels"]
    del save
    print("Training Set {0} {1}".format(train_dataset.shape, train_labels.shape))
    print("Test Set {0} {1}".format(test_dataset.shape, test_labels.shape))
test_labels[0:10, :]

Training Set (1688, 70, 100, 3) (1688, 3)
Test Set (300, 70, 100, 3) (300, 3)


array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.]])

In [14]:
# input_x = tf.placeholder(tf.float32, [None, img_flat_size], name = "input_x") 
input_layer = tf.placeholder(tf.float32, [None, rows, cols, img_channels], name = "input_layer")
y_true = tf.placeholder(tf.float32, [None, img_classes], name = "y_true")
y_true_cls = tf.argmax(y_true, dimension = 1)

In [48]:
x_pretty = pt.wrap(input_layer)
with pt.defaults_scope(activation_fn = tf.nn.relu):
    y_pred, loss = x_pretty.\
        conv2d(kernel = 5, depth = 16, name = "conv_layer_1").\
        max_pool(kernel = 2, stride = 2).\
        conv2d(kernel = 5, depth = 36, name = "conv_layer_2").\
        max_pool(kernel = 2, stride = 2).\
        flatten().\
        fully_connected(size = 128, name = "fc_layer_1").\
        softmax_classifier(num_classes = img_classes, labels = y_true)

In [49]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-4).minimize(loss)

In [50]:
y_pred_cls = tf.argmax(y_pred, dimension = 1)

In [51]:
correct_pred = tf.equal(y_true_cls, y_pred_cls)

In [52]:
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [53]:
train_batch_size = 128

In [55]:
num_steps = 150
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print("Variable Initialized")
    for step in range(num_steps):
        offset = (step * train_batch_size) % (train_labels.shape[0] - train_batch_size)
        print offset
        batch_train = train_dataset[offset:(offset + train_batch_size), :]
        batch_labels = train_labels[offset:(offset + train_batch_size), :]
        feed_dict_train = {
            input_layer: batch_train,
            y_true: batch_labels
        }
        
        _, l = sess.run([optimizer, loss], feed_dict = feed_dict_train)
        print("Loss : {0}".format(l))
        
    feed_dict_test = {
        input_layer: test_dataset,
        y_true: test_labels
    }
    true_value_class, acc, pred_value, pred_value_class = sess.run([y_true_cls, accuracy, y_pred, y_pred_cls], feed_dict = feed_dict_test)
    print("Accuracy {:.2%}".format(acc))
    print("True Value Class")
    print true_value_class[0:10]
    print("Pred Value")
    print pred_value[0:20]
    print("Pred Value Class")
    print pred_value_class

Variable Initialized
0
Loss : 1.18012523651
128
Loss : 1.12670111656
256
Loss : 1.16188037395
384
Loss : 1.14782977104
512
Loss : 1.11936569214
640
Loss : 1.09480285645
768
Loss : 1.09140694141
896
Loss : 1.0727263689
1024
Loss : 1.06990408897
1152
Loss : 1.08438026905
1280
Loss : 1.08298754692
1408
Loss : 1.07769155502
1536
Loss : 1.05682730675
104
Loss : 1.04254972935
232
Loss : 1.01842176914
360
Loss : 1.00757837296
488
Loss : 0.989777624607
616
Loss : 0.985747814178
744
Loss : 1.03270483017
872
Loss : 1.00496339798
1000
Loss : 1.02171111107
1128
Loss : 1.03797507286
1256
Loss : 1.06667089462
1384
Loss : 1.08234119415
1512
Loss : 1.01689302921
80
Loss : 0.996277213097
208
Loss : 0.908556699753
336
Loss : 0.904912471771
464
Loss : 0.897743940353
592
Loss : 0.905225515366
720
Loss : 0.949902713299
848
Loss : 0.978926181793
976
Loss : 0.989299595356
1104
Loss : 0.987731516361
1232
Loss : 1.04793953896
1360
Loss : 1.09000086784
1488
Loss : 1.03132498264
56
Loss : 0.973970293999
184
Loss